In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/clase_11/clase_11/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Práctica adicional sobre joins con pandas


Trabajaremos con un dataset de campañas de marketing. El mismo está representado por un conjunto de tablas relacionales:

* train: relaciona id de consumidores, cupones de descuento, campaña de marketing y contiene la variable target 'redemption_status', que dice si el cupón fue canjeado o no.

* campaign data: fecha de inicio y finalización de la campaña y tipo de campaña

* customer_demographics: rango etareo, estado civil, tamaño de la familia, nro de hijos, nivel de ingresos, propietario/inquilino/hipoteca

* coupon_item_mapping: cada cupón da beneficios sobre distintos items

* item_data: marca, tipo de marca y categoría del item

* customer_transaction_data: datos sobre las compras hechas por los consumidores. No sabemos en qué compra usaron el cupón de descuento, tal vez la promoción sirve para más de una compra.

<img src='../Data/Schema.png'>

<b>Origen del dataset:</b>

https://www.kaggle.com/vasudeva009/coupon-redemption-smote-feature-selection/data

In [2]:
import pandas as pd
from sklearn.impute import SimpleImputer

In [3]:
train=pd.read_csv('../Data/train.csv')
cust_dem=pd.read_csv('../Data/customer_demographics.csv')
trans=pd.read_csv('../Data/customer_transaction_data.csv')
coupon_item=pd.read_csv('../Data/coupon_item_mapping.csv')
item_data=pd.read_csv('../Data/item_data.csv')
campaign_data=pd.read_csv('../Data/campaign_data.csv')

---
#### Train dataframe

* Mirar el dataframe y chequear si hay valores nulos

In [4]:
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78369 entries, 0 to 78368
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   id                 78369 non-null  int64
 1   campaign_id        78369 non-null  int64
 2   coupon_id          78369 non-null  int64
 3   customer_id        78369 non-null  int64
 4   redemption_status  78369 non-null  int64
dtypes: int64(5)
memory usage: 3.0 MB


---
#### Customer demographics dataframe

* Chequear si hay nulos
* Mirar los dtypes
* Imputar los valores nulos por la moda de las columnas
* Llevar las columnas de variables categóricas a dtype 'category'

In [6]:
cust_dem.isnull().sum()

customer_id         0
age_range           0
marital_status    329
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64

In [7]:
cust_dem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     760 non-null    int64 
 1   age_range       760 non-null    object
 2   marital_status  431 non-null    object
 3   rented          760 non-null    int64 
 4   family_size     760 non-null    object
 5   no_of_children  222 non-null    object
 6   income_bracket  760 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 41.7+ KB


In [8]:
cust_dem.no_of_children.value_counts()

1     107
3+     60
2      55
Name: no_of_children, dtype: int64

In [9]:
cust_dem.marital_status.value_counts()

Married    317
Single     114
Name: marital_status, dtype: int64

In [10]:
cust_dem.no_of_children.fillna(1, inplace=True)

In [11]:
cust_dem.marital_status.fillna('Married', inplace = True)

In [12]:
cust_dem.iloc[:,1:]=cust_dem.iloc[:,1:].astype('category')

---
#### Transactions dataframe
* Chequear si hay nulos
* ver los dtypes
* dropear la columna 'dates'

In [13]:
trans.isnull().any().any()

False

In [14]:
trans.dtypes

date                object
customer_id          int64
item_id              int64
quantity             int64
selling_price      float64
other_discount     float64
coupon_discount    float64
dtype: object

In [15]:
trans.drop('date', axis=1, inplace=True)
trans

,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,1501,26830,1,35.26,-10.69,0.0
1,1501,54253,1,53.43,-13.89,0.0
2,1501,31962,1,106.50,-14.25,0.0
3,1501,33647,1,67.32,0.00,0.0
4,1501,48199,1,71.24,-28.14,0.0
...,...,...,...,...,...,...
1324561,1129,2777,1,284.60,-71.24,0.0
1324562,1129,2953,4,42.74,-28.50,0.0
1324563,1129,2971,6,64.12,-42.74,0.0
1324564,1129,46984,1,95.82,0.00,0.0


---
#### Coupon-Item dataframe
* chequear si hay nulos

In [16]:
coupon_item.isnull().any().any()

False

---
#### Item dataframe
* Chequear si hay nulos
* Llevar las columnas categóricas a dtype 'category'

In [17]:
item_data.isnull().any().any()

False

In [18]:
item_data.dtypes

item_id        int64
brand          int64
brand_type    object
category      object
dtype: object

In [19]:
item_data.iloc[:,1:] = item_data.iloc[:,2:].astype('category')

---
#### Campaign dataframe
* Chequear si hay valores nulos
* Llevar las columnas start_date y end_date a tipo TimeStamp
* Calcular el período de las promos como end_date - start_date
* Dropear start_date y end_date

In [20]:
campaign_data.isnull().any().any()

False

In [21]:
campaign_data.start_date.head()

0    21/10/13
1    21/10/13
2    07/09/13
3    08/10/13
4    16/09/13
Name: start_date, dtype: object

In [22]:
campaign_data.dtypes

campaign_id       int64
campaign_type    object
start_date       object
end_date         object
dtype: object

In [23]:
from datetime import datetime

s = campaign_data['start_date'].transform(lambda x : datetime.strptime(x, '%d/%m/%y'))
e = campaign_data['end_date'].transform(lambda x : datetime.strptime(x, '%d/%m/%y'))

campaign_data.loc[:,"start_date"]=s
campaign_data.loc[:,"end_date"]=e

In [24]:
campaign_data.columns

Index(['campaign_id', 'campaign_type', 'start_date', 'end_date'], dtype='object')

In [25]:
campaign_data['PeriodoPromo'] = campaign_data['end_date'] - campaign_data['start_date']

In [26]:
campaign_data.head()

,campaign_id,campaign_type,start_date,end_date,PeriodoPromo
0,24,Y,2013-10-21,2013-12-20,60 days
1,25,Y,2013-10-21,2013-11-22,32 days
2,20,Y,2013-09-07,2013-11-16,70 days
3,23,Y,2013-10-08,2013-11-15,38 days
4,21,Y,2013-09-16,2013-10-18,32 days


In [27]:
campaign_data.drop(["start_date","end_date"], inplace = True, axis=1)

In [28]:
campaign_data

,campaign_id,campaign_type,PeriodoPromo
0,24,Y,60 days
1,25,Y,32 days
2,20,Y,70 days
3,23,Y,38 days
4,21,Y,32 days
5,22,X,32 days
6,18,X,55 days
7,19,Y,32 days
8,17,Y,32 days
9,16,Y,32 days


---
#### Merge Dataframes
1. Hacer un merge entre train y campaign_data. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
2. Hacer un merge de 1. con customer_demographics. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
3. Chequear valores nulos en 2.
4. Imputamos valores faltantes en las columnas demográficas. Para eso, sampleamos con reposición de la distribución conjunta de atributos demográficos observados.
5. Merge con las transacciones promedio por sujeto:
* Hacer un groupby por consumidor en el dataframe de transaccions
* Agregar el valor medio a las variables continuas ('selling_price','quantity', etc)
* merge con el dataframe obtenido en 4. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
6. Merge con transacciones promedio por item
* a. Hacer un groupby por item en el dataframe de transaccions
* b. Calcular el valor medio de las variables continuas
* c. Hacer un merge con el dataframe 'coupon_item'
* d. Hacer un merge de c. con item_data ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
* e. Hacer un groupby del dataframe d. agrupando por 'coupon_id'. Para cada grupo calcular la moda de las variables categóricas 'brand','brand_type,'category'
* f. Hacer un groupby del dataframe d. agrupando por 'coupon_id'. Para cada grupo calcular la media de las variables continuas.
* g. Hacer un merge de d. con e. y luego con f. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
7. Guardar el dataframe resultante en un csv para la práctica de la clase.

In [29]:
train.head(3)

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0


In [30]:
campaign_data.head(3)

,campaign_id,campaign_type,PeriodoPromo
0,24,Y,60 days
1,25,Y,32 days
2,20,Y,70 days


In [31]:
uno=train.merge(campaign_data, how='left', on = 'campaign_id')
uno.shape

(78369, 7)

In [32]:
dos = uno.merge(cust_dem, how="left", on= 'customer_id')

In [33]:
dos.isnull().sum()

id                       0
campaign_id              0
coupon_id                0
customer_id              0
redemption_status        0
campaign_type            0
PeriodoPromo             0
age_range            34708
marital_status       34708
rented               34708
family_size          34708
no_of_children       34708
income_bracket       34708
dtype: int64

In [34]:
# Imputamos valores faltantes sampleando con reposicion de los datos observados
# De esta manera preservamos la estructura de correlaciones observada entre las columnas demográficas

impute_columns=['age_range','marital_status','rented','family_size','no_of_children','income_bracket']

null_mask=dos[impute_columns].isnull().all(axis=1)
random_samples=dos.loc[~null_mask,impute_columns].sample(n=null_mask.sum(),replace=True)
dos.loc[null_mask,impute_columns]=random_samples.values

dos.isnull().sum()

id                   0
campaign_id          0
coupon_id            0
customer_id          0
redemption_status    0
campaign_type        0
PeriodoPromo         0
age_range            0
marital_status       0
rented               0
family_size          0
no_of_children       0
income_bracket       0
dtype: int64

5. Merge con las transacciones promedio por sujeto:
* Hacer un groupby por consumidor en el dataframe de transaccions
* Agregar el valor medio a las variables continuas ('selling_price','quantity', etc)
* merge con el dataframe obtenido en 4. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?

In [35]:
trans.head(3)

,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,1501,26830,1,35.26,-10.69,0.0
1,1501,54253,1,53.43,-13.89,0.0
2,1501,31962,1,106.50,-14.25,0.0


In [36]:
customer_mean=trans.groupby('customer_id')[["quantity","selling_price","other_discount","coupon_discount"]].agg('mean')

In [37]:
customer_mean.reset_index()

,customer_id,quantity,selling_price,other_discount,coupon_discount
0,1,1.170802,94.001842,-16.250382,-2.019876
1,2,1.131265,102.864033,-16.830430,-0.595084
2,3,11.578723,103.617404,-22.714227,-3.091546
3,4,1.272727,154.423727,-13.305409,-0.404773
4,5,117.869949,130.827146,-13.657917,-0.114684
...,...,...,...,...,...
1577,1578,136.790549,104.750268,-18.625581,-0.980358
1578,1579,1.511188,125.954294,-22.934570,-0.291213
1579,1580,38.689981,112.336673,-12.697278,0.000000
1580,1581,282.318812,113.925228,-14.767089,0.000000


In [39]:
tres = pd.merge(dos, customer_mean, how='left', on='customer_id' )

In [40]:
tres.shape

(78369, 17)

6. Merge con transacciones promedio por item
* a. Hacer un groupby por item en el dataframe de transaccions
* b. Calcular el valor medio de las variables continuas
* c. Hacer un merge con el dataframe 'coupon_item'
* d. Hacer un merge de c. con item_data ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
* e. Hacer un groupby del dataframe d. agrupando por 'coupon_id'. Para cada grupo calcular la moda de las variables categóricas 'brand','brand_type,'category'
* f. Hacer un groupby del dataframe d. agrupando por 'coupon_id'. Para cada grupo calcular la media de las variables continuas.
* g. Hacer un merge de d. con e. y luego con f. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
7. Guardar el dataframe resultante en un csv para la práctica de la clase.

In [41]:
trans.head(3)

,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,1501,26830,1,35.26,-10.69,0.0
1,1501,54253,1,53.43,-13.89,0.0
2,1501,31962,1,106.50,-14.25,0.0


In [46]:
trans_mean=trans.groupby('item_id')[['quantity','selling_price','other_discount','coupon_discount']].agg('mean')

In [45]:
coupon_item.head(3)

,coupon_id,item_id
0,105,37
1,107,75
2,494,76


In [51]:
trans_mean.reset_index()

trans_mean_coupon_item = trans_mean.merge(coupon_item, how='inner', on='item_id')

In [52]:
trans_mean_coupon_item.shape

(92655, 6)

TERMIANAR TERMINAR TERMINAR